#Import These two lines of code to connect this ipynb file with google drive.This code will use google drive stored data.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Python Libraries 

In [2]:

import pandas as pd
import numpy as np
import datetime
import math 
from datetime import datetime, timedelta


#Loading All the data

In [3]:
prescriptions_data=pd.read_csv('/content/drive/MyDrive/ArianaProject/mimiciv/1.0/hosp/prescriptions.csv.gz')#Load all the rows of the data
propensity_data=pd.read_csv('/content/drive/MyDrive/ArianaProject/Transfusion/Data/propensity_matched_3days_or_more_mimic.csv')#Load all the rows of the data
ndc2_data=pd.read_csv('/content/drive/MyDrive/ArianaProject/Transfusion/Data/ndc2therapeutic.csv')#removed duplicates already

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#Prescriptions Full Data

In [4]:
prescriptions_data #Prescriptions Data Details

,subject_id,hadm_id,pharmacy_id,starttime,stoptime,drug_type,drug,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,17868682,22726960,73313910,2160-01-07 08:00:00,2160-01-07 21:00:00,MAIN,BuPROPion (Sustained Release),046238,5.910840e+08,150mg SR Tablet,NaN,300,mg,2,TAB,1.0,PO
1,17868682,22726960,16239987,2160-01-07 08:00:00,2160-01-08 16:00:00,MAIN,BuPROPion (Sustained Release),046238,5.910840e+08,150mg SR Tablet,NaN,150,mg,1,TAB,1.0,PO
2,17868682,22726960,16634804,2160-01-07 10:00:00,2160-01-07 16:00:00,MAIN,Aspirin,004380,9.044041e+08,81mg Tab,NaN,81,mg,1,TAB,1.0,PO
3,17868682,22726960,2697460,2160-01-07 14:00:00,2160-01-07 14:00:00,BASE,1/2 NS,001209,3.380043e+08,1000mL Bag,NaN,1000,mL,1000,mL,NaN,IV
4,17868682,22726960,1383959,2160-01-07 15:00:00,2160-01-08 16:00:00,MAIN,Pneumococcal Vac Polyvalent,048548,6.494300e+06,25mcg/0.5mL Vial,NaN,0.5,mL,1,VIAL,NaN,IM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17008048,17608878,22135112,65762013,2147-04-12 17:00:00,2147-04-13 12:00:00,MAIN,Fentanyl Citrate,41384.0,1.747800e+10,100mcg/2mL Amp,NaN,100-200,mcg,2-4,mL,1.0,IV
17008049,15906963,20494713,9933337,2164-11-17 12:00:00,2164-11-18 11:00:00,MAIN,Fentanyl Citrate,41384.0,4.099093e+08,100mcg/2mL Amp,NaN,25-100,mcg,0.5-2,mL,1.0,IV
17008050,11667755,29607134,31327446,2126-08-08 05:00:00,2126-08-08 14:00:00,MAIN,OxycoDONE (Immediate Release),4225.0,4.060553e+08,5mg Tablet,NaN,7.5-10,mg,1-2,TAB,NaN,PO
17008051,17961555,22879995,50647416,2171-01-24 05:00:00,2171-02-03 20:00:00,MAIN,Acetaminophen-Caff-Butalbital,4451.0,5.913369e+08,1 Tablet,NaN,1-2,TAB,1-2,TAB,NaN,PO


#This dataframe consists Working Features of Prescriptions data only

In [5]:

prescriptions_data_demo=prescriptions_data[['subject_id','starttime','stoptime','ndc','doses_per_24_hrs']].drop_duplicates()#Removed duplicates 
prescriptions_data_demo

,subject_id,starttime,stoptime,ndc,doses_per_24_hrs
0,17868682,2160-01-07 08:00:00,2160-01-07 21:00:00,5.910840e+08,1.0
1,17868682,2160-01-07 08:00:00,2160-01-08 16:00:00,5.910840e+08,1.0
2,17868682,2160-01-07 10:00:00,2160-01-07 16:00:00,9.044041e+08,1.0
3,17868682,2160-01-07 14:00:00,2160-01-07 14:00:00,3.380043e+08,NaN
4,17868682,2160-01-07 15:00:00,2160-01-08 16:00:00,6.494300e+06,NaN
...,...,...,...,...,...
17008048,17608878,2147-04-12 17:00:00,2147-04-13 12:00:00,1.747800e+10,1.0
17008049,15906963,2164-11-17 12:00:00,2164-11-18 11:00:00,4.099093e+08,1.0
17008050,11667755,2126-08-08 05:00:00,2126-08-08 14:00:00,4.060553e+08,NaN
17008051,17961555,2171-01-24 05:00:00,2171-02-03 20:00:00,5.913369e+08,NaN


#Propensity Full Data

In [6]:
propensity_data#Propensity Data Details

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,...,transfusion_icd_version,Transfusion,gender,anchor_age,patient_age_binned,E70-E88,I30-I52,I10-I16,J96-J99,N17-N19
0,0,3972,3972,14015,18109135,27606736,2171-04-12 14:00:00,2171-04-17 15:50:00,NaN,ELECTIVE,...,9.0,True,M,82,8,NaN,NaN,NaN,NaN,NaN
1,1,3990,3990,14199,19953910,27467958,2146-05-27 12:33:00,2146-06-04 12:10:00,NaN,DIRECT EMER.,...,9.0,True,M,56,5,NaN,NaN,NaN,NaN,NaN
2,2,4000,4000,14370,10953601,20359691,2167-09-28 22:52:00,2167-10-03 14:18:00,NaN,EU OBSERVATION,...,9.0,True,M,79,7,NaN,NaN,NaN,NaN,NaN
3,3,4362,4362,20056,19704964,21790335,2140-03-09 14:44:00,2140-03-15 13:00:00,NaN,URGENT,...,9.0,True,F,34,3,NaN,NaN,NaN,NaN,NaN
4,4,4363,4363,20057,15572065,27912170,2156-01-16 06:01:00,2156-02-24 15:34:00,NaN,ELECTIVE,...,10.0,True,M,48,4,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7799,7799,187088,187088,372867,18600365,26184092,2187-12-31 19:40:00,2188-01-13 14:00:00,NaN,DIRECT EMER.,...,NaN,False,F,64,6,NaN,NaN,NaN,NaN,NaN
7800,7800,251266,251266,500180,12203543,22233932,2156-05-31 14:15:00,2156-06-06 14:40:00,NaN,SURGICAL SAME DAY ADMISSION,...,NaN,False,M,58,5,NaN,NaN,NaN,NaN,NaN
7801,7801,13361,13361,36261,10101070,29592610,2153-12-24 04:22:00,2153-12-31 17:50:00,NaN,EW EMER.,...,NaN,False,M,85,8,NaN,NaN,NaN,NaN,NaN
7802,7802,19098,19098,44608,17420586,23714795,2129-11-24 14:30:00,2129-11-28 16:30:00,NaN,EW EMER.,...,NaN,False,F,70,7,NaN,NaN,NaN,NaN,NaN


#This dataframe consists Working Features of Propensity data only

In [7]:

propensity_data_demo=propensity_data[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates()#Removed duplicates
propensity_data_demo

,subject_id,hadm_id,admittime,dischtime
0,18109135,27606736,2171-04-12 14:00:00,2171-04-17 15:50:00
1,19953910,27467958,2146-05-27 12:33:00,2146-06-04 12:10:00
2,10953601,20359691,2167-09-28 22:52:00,2167-10-03 14:18:00
3,19704964,21790335,2140-03-09 14:44:00,2140-03-15 13:00:00
4,15572065,27912170,2156-01-16 06:01:00,2156-02-24 15:34:00
...,...,...,...,...
7799,18600365,26184092,2187-12-31 19:40:00,2188-01-13 14:00:00
7800,12203543,22233932,2156-05-31 14:15:00,2156-06-06 14:40:00
7801,10101070,29592610,2153-12-24 04:22:00,2153-12-31 17:50:00
7802,17420586,23714795,2129-11-24 14:30:00,2129-11-28 16:30:00


In [8]:
propensity_data_demo['subject_id'].isna().sum()#Checking any subject_id is missing or not

0

#ndc2 full data

In [9]:

ndc2_data=ndc2_data.drop_duplicates()#Removed duplicates
ndc2_data=ndc2_data.dropna(subset=['MED_THERAPEUTIC_CLASS_DESCRIPTION'])
ndc2_data

,NDC_MEDICATION_CODE,MED_THERAPEUTIC_CLASS_DESCRIPTION
0,904585261,ANTIHISTAMINES
1,115703301,HORMONES
2,23114501,AUTONOMIC DRUGS
3,641602110,GASTROINTESTINAL
4,115148361,ANTINEOPLASTICS
...,...,...
3251,186037220,ANTIASTHMATICS
3253,409508216,ANTIBIOTICS
3255,85132801,ANTIFUNGALS
3269,904552261,CARDIOVASCULAR


In [10]:

len(ndc2_data['MED_THERAPEUTIC_CLASS_DESCRIPTION'].unique())#Checking how many class in total

41

In [11]:
ndc2_data

,NDC_MEDICATION_CODE,MED_THERAPEUTIC_CLASS_DESCRIPTION
0,904585261,ANTIHISTAMINES
1,115703301,HORMONES
2,23114501,AUTONOMIC DRUGS
3,641602110,GASTROINTESTINAL
4,115148361,ANTINEOPLASTICS
...,...,...
3251,186037220,ANTIASTHMATICS
3253,409508216,ANTIBIOTICS
3255,85132801,ANTIFUNGALS
3269,904552261,CARDIOVASCULAR


#First merge between prescriptions data and ndc2 data

In [12]:

first_merge=pd.merge(prescriptions_data_demo,ndc2_data,left_on='ndc',right_on='NDC_MEDICATION_CODE',how='left')#Merging 'MED_THERAPEUTIC_CLASS_DESCRIPTION' column with prescriptions_data
first_merge_1=first_merge.dropna(subset=['MED_THERAPEUTIC_CLASS_DESCRIPTION'])#Droping 'MED_THERAPEUTIC_CLASS_DESCRIPTION' column's nan value only
first_merge_2= first_merge_1.drop("NDC_MEDICATION_CODE", axis='columns')
first_merge_2



,subject_id,starttime,stoptime,ndc,doses_per_24_hrs,MED_THERAPEUTIC_CLASS_DESCRIPTION
2,17868682,2160-01-07 10:00:00,2160-01-07 16:00:00,904404073.0,1.0,ANTIPLATELET DRUGS
3,17868682,2160-01-07 14:00:00,2160-01-07 14:00:00,338004304.0,NaN,ELECT/CALORIC/H2O
4,17868682,2160-01-07 15:00:00,2160-01-08 16:00:00,6494300.0,NaN,BIOLOGICALS
7,17868682,2160-01-07 15:00:00,2160-01-07 16:00:00,409176230.0,NaN,ANALGESICS
9,17868682,2160-01-07 15:00:00,2160-01-08 16:00:00,121176130.0,NaN,GASTROINTESTINAL
...,...,...,...,...,...,...
16178027,12683989,2121-02-06 17:00:00,2121-02-07 06:00:00,703040402.0,1.0,UNCLASSIFIED DRUG PRODUCTS
16178028,19871887,2144-06-09 15:00:00,2144-06-10 07:00:00,703040402.0,1.0,UNCLASSIFIED DRUG PRODUCTS
16178158,15906963,2164-11-17 12:00:00,2164-11-18 11:00:00,409909332.0,1.0,ANALGESICS
16178159,11667755,2126-08-08 05:00:00,2126-08-08 14:00:00,406055262.0,NaN,ANALGESICS


#Second Merge between First merge data and propensity data

In [13]:

second_merge=pd.merge(first_merge_2,propensity_data_demo,on='subject_id',how='left')#Merging first_merge_2 dataframe with propensity_data using subject_id
second_merge_1=second_merge.dropna(subset=['admittime'])#Droping 'admittime' column's nan value only
second_merge_1['starttime'] =  pd.to_datetime(second_merge_1['starttime'], infer_datetime_format=True)#Converting this column into datetime
second_merge_1['stoptime'] =  pd.to_datetime(second_merge_1['stoptime'], infer_datetime_format=True)#Converting this column into datetime
second_merge_1['admittime'] =  pd.to_datetime(second_merge_1['admittime'], infer_datetime_format=True)#Converting this column into datetime
second_merge_1['dischtime'] =  pd.to_datetime(second_merge_1['dischtime'], infer_datetime_format=True)#Converting this column into datetime
base_data=second_merge_1.copy()#Storing data to a new dataframe
base_data

<ipython-input-13-09f3fe16acec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_merge_1['starttime'] =  pd.to_datetime(second_merge_1['starttime'], infer_datetime_format=True)#Converting this column into datetime
<ipython-input-13-09f3fe16acec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_merge_1['stoptime'] =  pd.to_datetime(second_merge_1['stoptime'], infer_datetime_format=True)#Converting this column into datetime
<ipython-input-13-09f3fe16acec>:5: SettingWithCopyWarning: 
A value is t

,subject_id,starttime,stoptime,ndc,doses_per_24_hrs,MED_THERAPEUTIC_CLASS_DESCRIPTION,hadm_id,admittime,dischtime
32,12315540,2172-08-13 15:00:00,2172-08-14 14:00:00,409672924.0,1.0,ELECT/CALORIC/H2O,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00
33,12315540,2172-08-13 15:00:00,2172-08-14 14:00:00,409672924.0,1.0,ELECT/CALORIC/H2O,22831660.0,2173-03-26 20:33:00,2173-04-01 17:50:00
34,12315540,2172-08-16 09:00:00,2172-08-16 20:00:00,574705050.0,NaN,GASTROINTESTINAL,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00
35,12315540,2172-08-16 09:00:00,2172-08-16 20:00:00,574705050.0,NaN,GASTROINTESTINAL,22831660.0,2173-03-26 20:33:00,2173-04-01 17:50:00
36,12315540,2172-11-06 10:00:00,2172-11-07 16:00:00,904272561.0,1.0,ANTIBIOTICS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00
...,...,...,...,...,...,...,...,...,...
6891128,10805461,2148-06-12 23:00:00,2148-06-13 11:00:00,703040402.0,1.0,UNCLASSIFIED DRUG PRODUCTS,27104474.0,2142-04-25 14:50:00,2142-04-30 13:30:00
6891138,15895770,2151-04-10 13:00:00,2151-04-10 23:00:00,703040402.0,1.0,UNCLASSIFIED DRUG PRODUCTS,28093636.0,2142-07-01 22:50:00,2142-07-06 18:16:00
6891139,15895770,2151-04-10 13:00:00,2151-04-11 08:00:00,703040402.0,0.0,UNCLASSIFIED DRUG PRODUCTS,28093636.0,2142-07-01 22:50:00,2142-07-06 18:16:00
6891151,15906963,2164-11-17 12:00:00,2164-11-18 11:00:00,409909332.0,1.0,ANALGESICS,20494713.0,2164-11-10 00:41:00,2164-11-28 14:30:00


#This snippet is for makeing Timeperiod condition

In [14]:
# base_data['dif']=base_data['admittime']-base_data['starttime']#Creating new column for giving Timeperiod conditions
# base_data['Timeperiod']=np.nan#Creating and assigning null value to Timeperiod column
# base_data.loc[(base_data['dif'] > timedelta(days =0)) & (base_data['dif'] <= timedelta(days = 180) ),['Timeperiod']]='T1'#Condition for T1
# base_data.loc[(base_data['dif'] > timedelta(days =180)) & (base_data['dif'] <= timedelta(days = 365) ),['Timeperiod']]='T2'#Condition for T2
# base_data.loc[(base_data['dif'] <= timedelta(days =0)) & (base_data['dif']>=timedelta(days = -2) ),['Timeperiod']]='D1'#Condition for D1
# final_base_data=base_data.dropna(subset=['Timeperiod'])#Droping 'Timeperiod' column's nan value only
# final_base_data

#This portion excludes timeperiod and implements new condition

In [15]:
base_data['Timeperiod']=np.nan#Creating and assigning null value to Timeperiod column
base_data.loc[(base_data['starttime'] >= base_data['admittime']) & (base_data['starttime'] <= base_data['dischtime']),['Timeperiod']]='Valid' #valid when starttime is in between admittime and dischtime
final_base_data=base_data.dropna(subset=['Timeperiod'])#Droping 'Timeperiod' column's nan value only
final_base_data['ndc']=1
final_base_data

<ipython-input-15-0c8187d2a345>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_base_data['ndc']=1


,subject_id,starttime,stoptime,ndc,doses_per_24_hrs,MED_THERAPEUTIC_CLASS_DESCRIPTION,hadm_id,admittime,dischtime,Timeperiod
110,12315540,2172-12-25 14:00:00,2172-12-27 14:00:00,1,3.0,AUTONOMIC DRUGS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00,Valid
112,12315540,2172-12-27 20:00:00,2172-12-30 15:00:00,1,3.0,AUTONOMIC DRUGS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00,Valid
114,12315540,2172-12-29 20:00:00,2173-01-01 10:00:00,1,2.0,ANTIBIOTICS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00,Valid
116,12315540,2172-12-31 20:00:00,2173-01-01 12:00:00,1,3.0,AUTONOMIC DRUGS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00,Valid
118,12315540,2173-01-01 14:00:00,2173-01-19 08:00:00,1,3.0,AUTONOMIC DRUGS,27448605.0,2172-12-25 10:28:00,2173-01-29 15:00:00,Valid
...,...,...,...,...,...,...,...,...,...,...
6890982,17707918,2162-06-07 20:00:00,2162-06-08 13:00:00,1,4.0,HORMONES,23601930.0,2162-06-06 21:27:00,2162-06-15 17:08:00,Valid
6890983,17707918,2162-06-07 20:00:00,2162-06-11 19:00:00,1,4.0,HORMONES,23601930.0,2162-06-06 21:27:00,2162-06-15 17:08:00,Valid
6891005,18275245,2128-04-18 15:00:00,2128-04-18 15:00:00,1,4.0,HORMONES,22999142.0,2128-04-16 17:04:00,2128-05-04 00:00:00,Valid
6891151,15906963,2164-11-17 12:00:00,2164-11-18 11:00:00,1,1.0,ANALGESICS,20494713.0,2164-11-10 00:41:00,2164-11-28 14:30:00,Valid


In [16]:
final_base_data['Timeperiod'].value_counts()

Valid    234738
Name: Timeperiod, dtype: int64

In [17]:
#final_base_data['Timeperiod'].value_counts()#T1,T2 and D1 counts
len(final_base_data['subject_id'].unique())#Checking how many subject_id in total

6905

#Making Pivot Table for Expected Result

In [21]:
table = pd.pivot_table(final_base_data, values='ndc', index=['subject_id','hadm_id','admittime','dischtime'],columns=['MED_THERAPEUTIC_CLASS_DESCRIPTION'], aggfunc='sum')
table

,,,MED_THERAPEUTIC_CLASS_DESCRIPTION,ANALGESICS,ANESTHETICS,ANTIARTHRITICS,ANTIASTHMATICS,ANTIBIOTICS,ANTICOAGULANTS,ANTIDOTES,ANTIFUNGALS,ANTIHISTAMINES,ANTIHYPERGLYCEMICS,...,IMMUNOSUPPRESSANTS,MUSCLE RELAXANTS,PRE-NATAL VITAMINS,PSYCHOTHERAPEUTIC DRUGS,SEDATIVE/HYPNOTICS,SKIN PREPS,SMOKING DETERRENTS,THYROID PREPS,UNCLASSIFIED DRUG PRODUCTS,VITAMINS
subject_id,hadm_id,admittime,dischtime,,,,,,,,,,,,,,,,,,,,,
10002013,21763296.0,2165-11-23 08:19:00,2165-11-26 15:40:00,1.0,1.0,NaN,2.0,2.0,2.0,NaN,NaN,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004606,28731738.0,2159-04-05 01:58:00,2159-04-11 19:05:00,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10005866,20364112.0,2149-10-01 18:56:00,2149-10-25 18:50:00,18.0,3.0,NaN,NaN,6.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10006277,25610553.0,2176-06-07 23:15:00,2176-06-13 11:30:00,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10010066,22198822.0,2173-11-04 00:10:00,2173-11-20 18:00:00,6.0,NaN,NaN,NaN,2.0,3.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19997473,27787494.0,2173-09-11 00:53:00,2173-10-02 15:50:00,6.0,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997911,27144120.0,2196-09-16 23:53:00,2196-09-22 14:56:00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN
19998591,24349193.0,2185-07-03 20:20:00,2185-08-03 14:42:00,3.0,NaN,NaN,1.0,10.0,9.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,6.0,NaN,NaN,2.0,NaN,NaN


In [22]:
final_df = pd.DataFrame(table.to_records())
final_df

,subject_id,hadm_id,admittime,dischtime,ANALGESICS,ANESTHETICS,ANTIARTHRITICS,ANTIASTHMATICS,ANTIBIOTICS,ANTICOAGULANTS,...,IMMUNOSUPPRESSANTS,MUSCLE RELAXANTS,PRE-NATAL VITAMINS,PSYCHOTHERAPEUTIC DRUGS,SEDATIVE/HYPNOTICS,SKIN PREPS,SMOKING DETERRENTS,THYROID PREPS,UNCLASSIFIED DRUG PRODUCTS,VITAMINS
0,10002013,21763296.0,2165-11-23 08:19:00,2165-11-26 15:40:00,1.0,1.0,NaN,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10004606,28731738.0,2159-04-05 01:58:00,2159-04-11 19:05:00,5.0,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10005866,20364112.0,2149-10-01 18:56:00,2149-10-25 18:50:00,18.0,3.0,NaN,NaN,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10006277,25610553.0,2176-06-07 23:15:00,2176-06-13 11:30:00,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10010066,22198822.0,2173-11-04 00:10:00,2173-11-20 18:00:00,6.0,NaN,NaN,NaN,2.0,3.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7775,19997473,27787494.0,2173-09-11 00:53:00,2173-10-02 15:50:00,6.0,NaN,NaN,NaN,1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7776,19997911,27144120.0,2196-09-16 23:53:00,2196-09-22 14:56:00,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN
7777,19998591,24349193.0,2185-07-03 20:20:00,2185-08-03 14:42:00,3.0,NaN,NaN,1.0,10.0,9.0,...,NaN,NaN,NaN,4.0,6.0,NaN,NaN,2.0,NaN,NaN
7778,19999297,21439025.0,2162-08-14 23:55:00,2162-08-23 04:16:00,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
